In [1]:
library(qs)
library(Seurat)
library(ggplot2)

qs 0.25.7

Attaching SeuratObject



In [1]:
manifest = read.table("/mnt/accessory/seq_data/calico/Calico_Macosko_PD_GSA_Terra_Manifest.tsv", sep='\t',header=T,skip=0)
manifest$donor_id = sapply(manifest$entity.sample_id, function(x) gsub("_1$", "", x))
manifest$participant_id = manifest$collaborator_participant_id
manifest$age = manifest$Age 
manifest$sex = manifest$Sex
manifest$case_control = ifelse(manifest$Disease == "Control", "ctr", "pd")
manifest = manifest[, c('donor_id', 'participant_id', 'age', 'sex', 'case_control')]


In [4]:
data_seurat_nurr_all = qread("/mnt/accessory/seq_data/calico/seurat_nurr_merged_initial_harmonized_20240130.qs")
data_seurat_dapi = readRDS("/mnt/accessory/seq_data/calico/seurat_dapi_merged_initial_harmonized_20240119.rds")

In [5]:
nurr_markers = readRDS("/mnt/accessory/seq_data/calico/markers_nurr_initial_harmonized_20240130")
dapi_markers = readRDS("/mnt/accessory/seq_data/calico/markers_dapi_initial_harmonized_20240119")

Warning message in gzfile(file, "rb"):
“cannot open compressed file '/mnt/accessory/seq_data/calico/markers_nurr_initial_harmonized_20240130', probable reason 'No such file or directory'”


ERROR: Error in gzfile(file, "rb"): cannot open the connection


In [6]:
manifest

donor_id,participant_id,age,sex,case_control
<chr>,<chr>,<int>,<chr>,<chr>
206954930010_R05C02,PDC091,85,Male,ctr
206954930010_R09C01,PDC164,NA,Unknown,ctr
206954930011_R11C02,PDC165,NA,Unknown,ctr
206954930060_R11C02,PDC110,94,Female,ctr
206954930093_R05C02,PDC094,47,Female,ctr
207762950055_R07C02,PDC094,47,Female,ctr
207762950055_R11C01,2019-092,82,Female,pd
207762950086_R02C01,UnknownB,NA,Unknown,pd
207762950108_R10C02,UnknownC,NA,Unknown,pd


In [10]:
unique(read.table("/mnt/accessory/seq_data/calico/Calico_Macosko_PD_GSA_Terra_Manifest.tsv", sep='\t',header=T,skip=0)$Disease)

[1] "Control"                              
[2] "Parkinson&apos;s disease"             
[3] ""                                     
[4] "Parkinsons"                           
[5] "Parkinson&apos;s disease and dementia"